In [1]:
# PAN16

In [2]:
import os
from argparse import ArgumentParser
from sklearn.externals import joblib
from tictacs import from_recipe
from pan import ProfilingDataset
import dill
import cPickle as pickle
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix


infolder = "../DATA/pan16-author-profiling-training-dataset-2016-04-25/pan16-author-profiling-training-dataset-english-2016-02-29/"
outfolder = "models/"
print('Loading dataset->Grouping User texts.\n')
dataset = ProfilingDataset(infolder)
print('Loaded {} users...\n'.format(len(dataset.entries)))
# get config
config = dataset.config
tasks = config.tasks
print('\n--------------- Thy time of Running ---------------')
for task in tasks:
    print('Learning to judge %s..' % task)
    # load data
    X, y = dataset.get_data(task)

Loading dataset->Grouping User texts.

Loaded 436 users...


--------------- Thy time of Running ---------------
Learning to judge age..
Learning to judge gender..


### Instance Based

In [ ]:
from pan import ProfilingDataset, createDocProfiles, create_target_prof_trainset
from pan import preprocess

task = 'gender'
docs = createDocProfiles(dataset)
X, y = create_target_prof_trainset(docs, task)
print len(X)
X = preprocess.preprocess(X)

### Profile Based

In [3]:
#reload(preprocess)
#reload(features)
from pan import features
from pan import preprocess
X, y = dataset.get_data('age')
#X, y = dataset.get_data('gender')
print len(X)
#print X[0]
X = preprocess.preprocess(X)
#print "$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"
#print X[0]

436


# Models

### AGE

In [4]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from pan.features import SOA_Model2

grams3 = TfidfVectorizer(analyzer='word', ngram_range=[2,2], max_features=5000, stop_words='english')
soa = features.SOA_Model2(max_df=1.0, min_df=1, tokenizer_var='sklearn', max_features=None)
combined = FeatureUnion([('3grams', grams3), ('soa', soa)])
svm = SVC(kernel='rbf', C=10, gamma=1, class_weight='balanced', probability=False)
pipe = Pipeline([('3grams',grams3), ('svm', svm)])
pipe.steps

[('3grams',
  TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
          dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
          lowercase=True, max_df=1.0, max_features=5000, min_df=1,
          ngram_range=[2, 2], norm=u'l2', preprocessor=None, smooth_idf=True,
          stop_words='english', strip_accents=None, sublinear_tf=False,
          token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('svm', SVC(C=10, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False))]

### SOAC

In [5]:
soac = features.SOAC_Model2(max_df=1.0, min_df=1, tokenizer_var='sklearn', max_features=None)
svm = SVC(kernel='rbf', C=0.1, gamma=1, class_weight='balanced', probability=False)
#combined = FeatureUnion([('count_tokens', countTokens), ('count_hash', countHash),
#                         ('count_urls', countUrls), ('count_replies', countReplies), 
#                          ('soa', soa), ('soac', soac)])+
#combined = FeatureUnion([('count_tokens', countTokens), ('count_hash', countHash),
#                         ('count_urls', countUrls), ('count_replies', countReplies)])
pipe1 = Pipeline([('soac',soac), ('svm', svm)])
pipe1.steps

[('soac', SOAC_Model2(max_df=1.0, max_features=None, min_df=1, thres=0.1,
        tokenizer_var='sklearn')),
 ('svm', SVC(C=0.1, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False))]

### LSI

In [6]:
from pan.features import LSI_Model
LSImodel = LSI_Model(num_topics=25, max_df=1.0, min_df=2, max_features=None)
svm = SVC(kernel='rbf', C=10, gamma=1, class_weight='balanced', probability=False)
#pipe2 = Pipeline([('counts',combined), ('svm', svm)])
pipe2 = Pipeline([('LSI',LSImodel), ('svm', svm)])

### LDA

In [5]:
from pan.features import LDA
LDAmodel = LDA(lib='mallet', num_topics=90)
svm = SVC(kernel='rbf', C=100, gamma=1, class_weight='balanced', probability=False)
#pipe2 = Pipeline([('counts',combined), ('svm', svm)])
pipe2 = Pipeline([('LDA', LDAmodel), ('svm', svm)])
pipe2

mallet


Pipeline(steps=[('LDA', LDA(lib='mallet', num_topics=90)), ('svm', SVC(C=100, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

# Train

In [7]:
def print_overlaps(predictions, names, verbose=True):
    N = len(names)
    res = numpy.zeros([N,N])
    temp = numpy.zeros([N,N])
    for i in range(0, N):
        for j in range(i+1, N):
            temp[i,j] = len([m for l, m in enumerate(predictions[i]) if (m==predictions[j][l] and m==predictions[N-1][l])])/float(len(predictions[0]))
            res[i,j] = len([(k,v) for k,v in zip(predictions[i], predictions[j]) if k==v])/float(len(predictions[0]))
            if verbose:
                print "%s - %s : %0.3f  overlap | ground-truth coverage: %0.3f" % (names[i],  names[j], 100*res[i,j], 100*temp[i,j])
    return  [res, temp]

In [ ]:
reload(combinations)
combinations.

In [10]:
import sys
sys.path.insert(0, "../EnsembleDiversityTests/")
import combinations
import copy
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,precision_recall_fscore_support, classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.grid_search import GridSearchCV
from pan.features import Metaclassifier
import time
import numpy

import warnings
warnings.filterwarnings('ignore')

#pipe = Pipeline([('3grams',grams3), ('svm', svm)])
#pipe1 = Pipeline([('soac',soac), ('svm', svm)])
#pipe2 = Pipeline([('soa',soa), ('svm', svm)])

### AGE ###
#eclf = VotingClassifier(estimators=[("0", pipe), ('1', pipe1), ("2", pipe2)], voting='soft')
#eclfh = VotingClassifier(estimators=[("0", pipe), ('1', pipe1), ("2", pipe2)], voting='hard')
#models = [pipe,pipe1,pipe2,eclf, eclfh]
#model_names = ['3grams', 'soac', 'lda', 'voting', 'votingh']

# Base Models
base_models = [pipe, pipe1, pipe2]
base_model_names = ['3grams', 'soac', 'lsi']

# Meta Voting Models
eclf = VotingClassifier(estimators=[("0", pipe), ('1', pipe1), ('2', pipe2)], voting='soft')
eclfh = VotingClassifier(estimators=[("0", pipe), ('1', pipe1), ('2', pipe2)], voting='hard')
voting_dic = {'votingf':eclf, 'votingh':eclfh}
combinator_names = ['majority', 'weights', 'accuracy', 'optimal']
#meta_models_names = ['votingf', 'votingh', 'space3', 'meta'] + combinator_names
meta_models_names = ['space3', 'OLA', 'LCA', 'KNE', 'KNU'] + combinator_names
#meta_models_names = []
## all_models ##
all_models_names = base_model_names + meta_models_names


#eclf = VotingClassifier(estimators=[("0", pipe), ('1', pipe1)], voting='soft')
#eclfh = VotingClassifier(estimators=[("0", pipe), ('1', pipe1)], voting='hard')
#models = [pipe,pipe1,eclf, eclfh]
#model_names = ['3grams', 'soac', 'voting', 'votingh']

results = {'over':[]}
for name in all_models_names:
    results[name] = {'pred': [], 'conf': [], 'rep': [], 'acc': []}

num_folds = 4
train_split = 0.3
meta_split = 0.5
cv_rounds = 5
seeds = list(xrange(cv_rounds))
t0 = time.time()
t1 = t0
for j in xrange(cv_rounds):
    X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=train_split, stratify=y, random_state=seeds[j])
    for i, x in enumerate(X_train):
        if len(x)==0:
            X_train.remove(x)
            y_train.remove(y_train[i])
    for i, x in enumerate(X_cv):
        if len(x)==0:
            X_cv.remove(x)
            y_cv.remove(y_cv[i])
    if meta_split > 0:
        X_meta, X_cv, y_meta, y_cv = train_test_split(X_cv, y_cv, test_size=meta_split, stratify=y_cv)
        print len(X_train), len(X_cv), len(X_meta), len(X_cv) + len(X_train) + len(X_meta), len(X)
    else:
        print len(X_train), len(X_cv), len(X_cv) + len(X_train) , len(X)
    trained_base_models = []
    predictions = []
    base_predictions_cv = []
    base_predictions_meta = []
    for i, model in enumerate(base_models):
        model.fit(X_train,y_train)
        trained_base_models.append(model)
        predict = model.predict(X_cv)
        predictions.append(predict)
        base_predictions_cv.append(predict)
        base_predictions_meta.append(model.predict(X_meta))
        results[base_model_names[i]]['pred'].append(predict)
        results[base_model_names[i]]['acc'].append(accuracy_score(y_cv, predict))
        results[base_model_names[i]]['conf'].append(confusion_matrix(y_cv, predict, labels=sorted(list(set(y)))))
        results[base_model_names[i]]['rep'].append(classification_report(y_cv, predict, labels=sorted(list(set(y)))))
    trained_all_models = copy.deepcopy(trained_base_models)
    for name in meta_models_names:
        #print name
        if name =='votingf' or name=='votingh':
            model = voting_dic[name]
            model.fit(X_train, y_train)
            predict = model.predict(X_cv)
        if name == 'space':
            models_for_space = {}
            cv_scores = []
            for i, base_trained_model in enumerate(trained_base_models):
                models_for_space[base_model_names[i]] = base_trained_model
                cv_scores.append(base_trained_model.score(X_meta, y_meta))
            model = combinations.SubSpaceEnsemble4_2(models_for_space, cv_scores, k=6, weights=[0.65,0.35,0.32,6], N_rand=10, rand_split=0.6)
            model.fit(X_meta, y_meta)
            predict = model.predict(X_cv)
        if name == 'space3':
            models_for_space = {}
            for i, base_trained_model in enumerate(trained_base_models):
                models_for_space[base_model_names[i]] = base_trained_model
            model = SubSpaceEnsemble3(models_for_space, k=5, weights= [2,1,3,0.6])
            model.fit(X_train, y_train)
            predict = model.predict(X_cv)
        if name == 'meta':
            model_dic = {}
            for i, base_trained_model in enumerate(trained_base_models):
                model_dic[base_model_names[i]] = base_trained_model
            model = Metaclassifier(models=model_dic, C=1.0, weights='balanced')
            model.fit(X_meta, y_meta)
            predict = model.predict(X_cv)
        if name == 'OLA' or name == 'LCA' or name== 'KNE' or name == 'KNU':
            models11 = {}
            models_tr11 = {}
            if name == 'LCA':
                for i, model1 in enumerate(trained_base_models[:]):
                    models11[base_model_names[i]] = model1
                    models_tr11[base_model_names[i]] = trained_base_models[i].steps[0][1]        
            else:   
                for i, model1 in enumerate(trained_base_models[:]):
                    models11[base_model_names[i]] = model1
                    models_tr11[base_model_names[i]] = trained_base_models[i].steps[0][1]        
            model = Neigbors_DS(models11, models_tr11, k=3, scheme=name, common_neigh=False)
            #print model
            model.fit(X_meta, y_meta)
            predict = model.predict(X_cv)
        if name in combinator_names:
            #print 'mpike'
            model = combinations.Combinator(scheme=name, weights= [1/float(len(base_predictions_meta)) for i in xrange(len(base_predictions_meta))])
            model.fit(base_predictions_meta, y_meta)
            predict = model.predict(base_predictions_cv)
        trained_all_models.append(model)
        predictions.append(predict)
        results[name]['pred'].append(predict)
        results[name]['acc'].append(accuracy_score(y_cv, predict))
        results[name]['conf'].append(confusion_matrix(y_cv, predict, labels=sorted(list(set(y)))))
        results[name]['rep'].append(classification_report(y_cv, predict, labels=sorted(list(set(y)))))
    print('Round %d took: %0.3f seconds') % (j, time.time()-t1)
    t1 = time.time()
print('Total time: %0.3f seconds') % (time.time()-t0)

303 65 64 432 436
Couldn't import dot_parser, loading of dot files will not be possible.
Fit took: 11.560 seconds
Predict took: 108.211 seconds
Fitting time 2.49
Fitting time 2.41
Fitting time 2.36
Fitting time 2.49
Round 0 took: 160.399 seconds
301 66 65 432 436
Fit took: 12.486 seconds
Predict took: 22.893 seconds
Fitting time 2.31
Fitting time 2.18
Fitting time 2.16
Fitting time 2.11
Round 1 took: 75.378 seconds
302 65 65 432 436
Fit took: 11.824 seconds
Predict took: 68.530 seconds
Fitting time 2.33
Fitting time 2.37
Fitting time 2.32
Fitting time 2.31
Round 2 took: 119.679 seconds
304 64 64 432 436
Fit took: 11.921 seconds
Predict took: 72.626 seconds
Fitting time 2.15
Fitting time 2.10
Fitting time 2.10
Fitting time 2.11
Round 3 took: 122.309 seconds
302 65 65 432 436
Fit took: 11.602 seconds
Predict took: 25.270 seconds
Fitting time 2.46
Fitting time 2.47
Fitting time 2.46
Fitting time 2.47
Round 4 took: 76.907 seconds
Total time: 554.673 seconds


In [46]:
for i, k in enumerate(results['lsi']['conf']):
    print k
    print results['lsi']['rep'][i]

[[ 0  0  4  0  0]
 [ 0  0 21  0  0]
 [ 0  0 27  0  0]
 [ 0  0 12  0  0]
 [ 0  0  1  0  0]]
             precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         4
      25-34       0.00      0.00      0.00        21
      35-49       0.42      1.00      0.59        27
      50-64       0.00      0.00      0.00        12
      65-xx       0.00      0.00      0.00         1

avg / total       0.17      0.42      0.24        65

[[ 0  0  4  0  0]
 [ 0  0 20  0  0]
 [ 0  0 27  0  0]
 [ 0  0 12  0  0]
 [ 0  0  1  0  0]]
             precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         4
      25-34       0.00      0.00      0.00        20
      35-49       0.42      1.00      0.59        27
      50-64       0.00      0.00      0.00        12
      65-xx       0.00      0.00      0.00         1

avg / total       0.18      0.42      0.25        64

[[ 0  0  4  0  0]
 [ 0  0 21  0  0]
 [ 0  0 28  0  0]
 [ 0  0 12  0  0

In [11]:
predict = trained_base_models[2].predict(X_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Accuracy : 0.425287356322
Confusion matrix :
 [[ 0  0  5  0  0]
 [ 0  0 28  0  0]
 [ 0  0 37  0  0]
 [ 0  0 16  0  0]
 [ 0  0  1  0  0]]
Classification report :
              precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         5
      25-34       0.00      0.00      0.00        28
      35-49       0.43      1.00      0.60        37
      50-64       0.00      0.00      0.00        16
      65-xx       0.00      0.00      0.00         1

avg / total       0.18      0.43      0.25        87



In [12]:
for name in all_models_names:
    print '%%%%%%%%%%%%%%%%  ' + name  + '  % %%%%%%%%%%%%%%%%%%%%%%%'
    print '#################################'
    mean_acc = 0
    mean_prec = 0
    mean_rec = 0
    mean_f1 = 0
    conf = numpy.zeros([5,5])
    for i in xrange(cv_rounds):
        mean_acc += results[name]['acc'][i]
        #print results[key]['report'][i].split('     ')
        mean_prec += float(results[name]['rep'][i].split('     ')[-4][2:])
        mean_rec += float(results[name]['rep'][i].split('     ')[-3][2:])
        mean_f1 += float(results[name]['rep'][i].split('     ')[-2][2:])
        conf += results[name]['conf'][i]
    mean_acc = mean_acc/float(cv_rounds)
    mean_prec = mean_prec/float(cv_rounds)
    mean_rec = mean_rec/float(cv_rounds)
    mean_f1 = mean_f1/float(cv_rounds)
    conf = conf/float(cv_rounds)
    print('Accuracy : {}'.format(mean_acc))
    print('Precision : {}'.format(mean_prec))
    print('Recall : {}'.format(mean_rec))
    print('F1 : {}'.format(mean_f1))
    print('Confusion matrix :\n {}'.format(conf))
    print '#################################'
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

%%%%%%%%%%%%%%%%  3grams  % %%%%%%%%%%%%%%%%%%%%%%%
#################################
Accuracy : 0.378505244755
Precision : 0.288
Recall : 0.376
F1 : 0.318
Confusion matrix :
 [[  0.2   1.4   2.    0.4   0. ]
 [  0.    6.8  13.8   0.2   0. ]
 [  0.    9.2  17.6   0.4   0. ]
 [  0.    3.4   8.6   0.    0. ]
 [  0.    0.    1.    0.    0. ]]
#################################
%%%%%%%%%%%%%%%%  soac  % %%%%%%%%%%%%%%%%%%%%%%%
#################################
Accuracy : 0.292671911422
Precision : 0.302
Recall : 0.294
F1 : 0.288
Confusion matrix :
 [[  0.6   0.8   1.6   0.8   0.2]
 [  3.    4.2  11.    2.    0.6]
 [  2.4   6.   12.4   4.2   2.2]
 [  2.    2.6   5.4   1.6   0.4]
 [  0.    0.    0.4   0.4   0.2]]
#################################
%%%%%%%%%%%%%%%%  lsi  % %%%%%%%%%%%%%%%%%%%%%%%
#################################
Accuracy : 0.418454254079
Precision : 0.174
Recall : 0.42
F1 : 0.244
Confusion matrix :
 [[  0.    0.    4.    0.    0. ]
 [  0.    0.   20.8   0.    0. ]
 [  0.    0.

In [16]:
import EnsembleDiversityTests
reload(EnsembleDiversityTests)
a = EnsembleDiversityTests.BaseClassifiers(base_predictions_cv, base_model_names, y_cv)
b = EnsembleDiversityTests.DiversityTests(base_predictions_cv, base_model_names, y_cv)
a.get_classification_report()
a.vis_flag = True
a.get_comparison_report()
b.print_report()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ CLASSIFICATION RESULTS PER MODEL~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
============================ Model: 3grams ==================================+
Accuracy : 0.369230769231
Confusion matrix :
 [[ 0  2  2  0  0]
 [ 0  7 14  0  0]
 [ 1  9 17  0  0]
 [ 0  4  8  0  0]
 [ 0  0  1  0  0]]
Classification report :
              precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         4
      25-34       0.32      0.33      0.33        21
      35-49       0.40      0.63      0.49        27
      50-64       0.00      0.00      0.00        12
      65-xx       0.00      0.00      0.00         1

avg / total       0.27      0.37      0.31        65

============================ Model: soac ==================================+
Accuracy : 0.246153846154
Confusion matrix :
 [[ 0  1  0  3  0]
 [ 0  3  5 13  0]
 [ 0  2  4 21  0]
 [ 0  1  2  9  0]
 [ 0  0  0  1  0]]
Classification report :
              precision    recall  f1-score   sup

In [13]:
pipe2

Pipeline(steps=[('LSI', LSI_Model(max_df=1.0, max_features=None, min_df=2, num_topics=25)), ('svm', SVC(C=10, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [52]:
class XGBoostClassifier(BaseEstimator, TransformerMixin):

    def __init__(self, **params):

        self.clf = None
        # self.num_boost_round = 100
        self.labels = None
        self.params = params.copy()
        for name, label in params.iteritems():
            setattr(self, name, label)

    def fit(self, X, y, num_boost_round=None):

        import xgboost as xgb

        num_boost_round = num_boost_round or self.num_boost_round
        #print self.params
        #print self.__dict__
        #print num_boost_round
        self.label2num = dict((label, i) for i, label in enumerate(sorted(set(y))))
        self.params.update({'num_class': len(set(y))})
        #print type(self.params)
        #print self.params
        # self.params.update({'objective': 'multi:softprob', 
        #                     'num_class': len(set(y)), 
        #                     'eval_metric': 'merror', 
        #                     'nthread': -1, 
        #                     'learning_rate': 0.1, 
        #                     'n_estimators': 140, 
        #                     'max_depth': 5,
        #                     'min_child_weight': 1, 
        #                     'gamma': 0, 
        #                     'subsample': 0.8, 
        #                     'colsample_bytree': 0.8,
        #                     'scale_pos_weight': 1, 
        #                     'seed': 27})
        if y is None:
            raise ValueError('we need y labels to supervise-fit!')
        else:
            dtrain = xgb.DMatrix(X, label=[self.label2num[label] for label in y])
            self.clf = xgb.train(params=self.params, dtrain=dtrain, num_boost_round=self.num_boost_round)

    def predict(self, X):

        import numpy
        from xgboost import DMatrix

        num2label = dict((i, label) for label, i in self.label2num.items())
        dtest = DMatrix(X)
        Y = self.clf.predict(dtest)
        return numpy.array([num2label[i] for i in Y] )
        #Y = self.predict_proba(X)
        #y = numpy.argmax(Y, axis=1)
        #return numpy.array([num2label[i] for i in y])

    def predict_proba(self, X):

        from xgboost import DMatrix

        dtest = DMatrix(X)
        return self.clf.predict(dtest)

    def score(self, X, y_true):

        from sklearn.metrics import accuracy_score

        y_pred = self.predict(X)
        return accuracy_score(y_true, y_pred, normalize=True)

        # Y = self.predict_proba(X)
        # return 1 / logloss(y, Y)

    def get_params(self, deep=True):
        return self.params

    def set_params(self, **params):
        if 'num_boost_round' in params:
            self.num_boost_round = params.pop('num_boost_round')
        if 'objective' in params:
            del params['objective']
        self.params.update(params)
        return self

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split 
params = {
    'svm__C':[0.01, 0.1, 1, 10, 100],
    'svm__kernel':['rbf', 'linear'],
    'LSI__num_topics': [25,50,100, 150],
    'LSI__max_df': [1.0, 0.9, 0.8],
    'LSI__max_features': [None, 5000, 10000]
}
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.3, stratify=y, )
grid1 = GridSearchCV(pipe2, param_grid =params, cv=4, verbose=2, n_jobs=-1)
grid1.fit(X_train, y_train)
print(grid1.best_score_)
print(grid1.best_params_)
pipe2 = grid1.best_estimator_

In [34]:
predict = trained_base_models[2].predict(X_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Accuracy : 0.424242424242
Confusion matrix :
 [[ 0  0  4  0  0]
 [ 0  0 21  0  0]
 [ 0  0 28  0  0]
 [ 0  0 12  0  0]
 [ 0  0  1  0  0]]
Classification report :
              precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         4
      25-34       0.00      0.00      0.00        21
      35-49       0.42      1.00      0.60        28
      50-64       0.00      0.00      0.00        12
      65-xx       0.00      0.00      0.00         1

avg / total       0.18      0.42      0.25        66



In [53]:
#import pan.features
#reload(pan.features)
from sklearn.pipeline import Pipeline, FeatureUnion
#from pan.features import XGBoostClassifier
from sklearn.grid_search import GridSearchCV
estimator_list = []
for i, model in enumerate(trained_base_models):
    estimator_list.append((base_model_names[i], trained_base_models[i].steps[0][1]))
FU = FeatureUnion(estimator_list)


xgb_params = {'objective': 'multi:softmax',
                            'num_boost_round': 100,
                            'num_class': len(set(y)), 
                            'eval_metric': 'merror', 
                            'nthread': -1, 
                            'learning_rate': 0.1, 
                            'n_estimators': 140, 
                            'max_depth': 5,
                            'min_child_weight': 1, 
                            'gamma': 0, 
                            'subsample': 0.8, 
                            'colsample_bytree': 0.8,
                            'scale_pos_weight': 1, 
                            'seed': 27}
#XGB = XGBoostClassifier()
#XGB.set_params
params = {
    'XGBOOST__max_depth':list(numpy.arange(3,10,1)),
    'XGBOOST__gamma':[0, 0.1, 0.2],
    'XGBOOST__min_child_weight':range(1,6,2),
    'XGBOOST__learning_rate': list(numpy.arange(0.1, 0.3, 0.05)),
    'XGBOOST__reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
}
pipe_fu = Pipeline([('FU',FU), ('XGBOOST', XGBoostClassifier(**xgb_params))])
grid1 = GridSearchCV(pipe_fu, param_grid =params, cv=4, verbose=1)
grid1.fit(X_meta, y_meta)
print(grid1.best_score_)
print(grid1.best_params_)
pipe_fu = grid1.best_estimator_
predict = pipe_fu.predict(X_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:  4.6min
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed: 18.7min
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed: 42.3min
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed: 75.9min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed: 120.6min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed: 177.9min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed: 248.4min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed: 328.3min
[Parallel(n_jobs=1)]: Done 4049 tasks       | elapsed: 418.8min
[Parallel(n_jobs=1)]: Done 4999 tasks       | elapsed: 519.6min


Fitting 4 folds for each of 1260 candidates, totalling 5040 fits
0.40625
{'XGBOOST__max_depth': 3, 'XGBOOST__learning_rate': 0.25000000000000006, 'XGBOOST__min_child_weight': 3, 'XGBOOST__reg_alpha': 0.1, 'XGBOOST__gamma': 0}
Accuracy : 0.369230769231
Confusion matrix :
 [[ 1  0  3  0  0]
 [ 0  5 12  4  0]
 [ 1  9 16  1  0]
 [ 0  1  9  2  0]
 [ 0  0  1  0  0]]
Classification report :
              precision    recall  f1-score   support

      18-24       0.50      0.25      0.33         4
      25-34       0.33      0.24      0.28        21
      35-49       0.39      0.59      0.47        27
      50-64       0.29      0.17      0.21        12
      65-xx       0.00      0.00      0.00         1

avg / total       0.35      0.37      0.34        65



[Parallel(n_jobs=1)]: Done 5040 out of 5040 | elapsed: 524.0min finished


In [53]:
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer

counter = CountVectorizer()
parameters = {
                    'input': 'content',
                    'encoding': 'utf-8',
                    'decode_error': 'ignore',
                    'analyzer': 'word',
                    'stop_words': 'english',
                    # 'vocabulary':list(voc),
                    #'tokenizer': tokenization,
                    #'tokenizer': _twokenize.tokenizeRawTweetText,  # self.tokenization,
                    #'tokenizer': lambda text: _twokenize.tokenizeRawTweetText(nonan.sub(po_re.sub('', text))),
                    'max_df': 1.0,
                    'min_df': 1,
                    'max_features':None
                }
counter.set_params(**parameters)
X_train_truth = counter.fit_transform(X_train)
print X_train_truth.shape
num_folds = 5
params = {'n_neighbors':[i for i in xrange(1,20)], 'weights':['uniform', 'distance']}
gg = GridSearchCV(KNeighborsClassifier(), param_grid=params, n_jobs=-1, cv=num_folds, refit=True, verbose=1)
gg.fit(X_train_truth, y_train)
print(gg.best_score_)
print(gg.best_params_)
predict = gg.best_estimator_.predict(counter.transform(X_meta))
print counter.transform(X_meta).shape
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

(301, 102406)
Fitting 5 folds for each of 38 candidates, totalling 190 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 190 out of 190 | elapsed:    3.6s finished


0.511627906977
{'n_neighbors': 15, 'weights': 'uniform'}
(65, 102406)


ValueError: Found arrays with inconsistent numbers of samples: [65 66]

In [115]:
>>> from sklearn.neighbors import KNeighborsClassifier
>>> X = [[3], [0], [1], [4]]
>>> y = [0, 0, 1, 1]
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(X, y) 
neigh.kneighbors([0.5])

(array([[ 0.5,  0.5]]), array([[1, 2]]))

In [24]:
print results['LCA']['rep'][-1]
print results['OLA']['rep'][-1]

             precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         5
      25-34       0.32      0.36      0.34        28
      35-49       0.46      0.51      0.49        35
      50-64       0.14      0.12      0.13        16
      65-xx       0.00      0.00      0.00         1

avg / total       0.32      0.35      0.34        85

             precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         5
      25-34       0.32      0.36      0.34        28
      35-49       0.46      0.51      0.49        35
      50-64       0.14      0.12      0.13        16
      65-xx       0.00      0.00      0.00         1

avg / total       0.32      0.35      0.34        85



In [8]:
import sys
sys.path.insert(0, "../EnsembleDiversityTests/")

from EnsembleDiversityTests import DiversityTests, BaseClassifiers
from sklearn.base import BaseEstimator, TransformerMixin


class Neigbors_DS(BaseEstimator, TransformerMixin):
    
    """ Best model base on the predictions of the k-nearest neighbors. Many different schemes.
        Also, implements a common neighborhoud instead a per transformation one.
        
        Args:
            - scheme: String flag. Can be one of the following:
                - 'LCA': Local Class Accuracy
                - 'OLA': Overall Local Accuracy
                - 'KNE': K_Neighbors Elimination. Start from a k 
                - 'optimal': The optimal weights are found, this
                             is done by optimizing over the classification
                             error
                - weights: list or numpy.array(not sure?) containing as many
                             weights as the models in the ensemble
        Returns:
            - The  ensemble Model. Needs to be fitted for the encoding part
        
        """

    def __init__(self, models, models_tr, k= 5, scheme='LCA', common_neigh=False):

        if (not models) or (not models_tr):
            raise AttributeError('Models expexts a dictonary of models \
              containg the predictions of y_true for each classifier.\
              cv_score expects a list len(models.keys()) with the\
              cross validation scores of each model')
        else:
            self.models = models
            self.models_tr = models_tr
            self.k = k
            self.ind2names = {}
            for i, name in enumerate(models.keys()):
                self.ind2names[i] = name
            self.predictions = {}
            self.true = []
            self.trees = {}
            self.scheme = scheme
            self.common_neigh = common_neigh
            if common_neigh:
                from sklearn.feature_extraction.text import CountVectorizer
        
                self.counter = CountVectorizer()
                parameters = {
                        'input': 'content',
                        'encoding': 'utf-8',
                        'decode_error': 'ignore',
                        'analyzer': 'word',
                        'stop_words': 'english',
                        # 'vocabulary':list(voc),
                        #'tokenizer': tokenization,
                        #'tokenizer': _twokenize.tokenizeRawTweetText,  # self.tokenization,
                        #'tokenizer': lambda text: _twokenize.tokenizeRawTweetText(nonan.sub(po_re.sub('', text))),
                        'max_df': 1.0,
                        'min_df': 1,
                        'max_features':None
                    }
                self.counter.set_params(**parameters)
                self.gt_tree = None
            else:
                self.counter = None
            if self.scheme == 'LCA':
                self.predictor = self.predict_lca
            elif self.scheme == 'KNE':
                self.predictor = self.predict_kne
            elif self.scheme == 'OLA':
                self.predictor = self.predict_ola
            elif self.scheme == 'KNU':
                self.predictor = self.predict_knu
            else:
                self.predictor = self.predict_ola
                
    def fit(self, X_cv, y_true=None, weights=None):
        from sklearn.neighbors import BallTree
        from sklearn.metrics import accuracy_score
        import random
        import time

        if y_true is None:
            raise ValueError('we need y labels to supervise-fit!')
        else:
            t0 = time.time()
            predictions = []
            for name, model in self.models.iteritems():
                #predictions.append(model.predict(X_cv))
                # print len(predictions[-1])
                if self.common_neigh:
                    X_tr = self.counter.fit_transform(X_cv)
                    self.gt_tree = BallTree(X_tr.toarray(), leaf_size=20)
                else:
                    X_tr = self.models_tr[name].transform(X_cv)
                    if hasattr(X_tr, "toarray"):
                        self.trees[name] = BallTree(X_tr.toarray(), leaf_size=20)
                    else:
                        self.trees[name] = BallTree(X_tr, leaf_size=20)    
                self.predictions[name] = model.predict(X_cv)
            self.true = y_true
            print 'Fitting time %0.2f' % (time.time() - t0)

    def predict(self, X):
        # import time

        # print "PRedict"
        # print X.shape
        y_pred = []
        # t0 = time.time()
        for i, x in enumerate(X):
            # print 'True Sample: ' + y_real[i]
            y_pred.append(self.predictor(x))
        # print('Predict took: %0.3f seconds') % (time.time()-t0)
        return y_pred

    def score(self, X, y, sample_weight=None):

        from sklearn.metrics import accuracy_score
        return accuracy_score(y, self.predict(X), normalize=True)
        # return self.svc.score(self.transform_to_y(X), y, sample_weight)


    def predict_lca(self, sample):
        preds = []
        for name, model in self.models.iteritems():
            preds.append(model.predict([sample])[0])
#         print 'Preds: ' + str(preds)
        if len(set(preds))==1:
#             print 'Unanimous Decision: ' + str(preds[0])
#             print '='*50
            return preds[0]
        else:
            lca = [0 for pred in preds]
            model_ind = 0
            for name, model in self.models.iteritems():
                # print 'Model: ' + name
                sample_trans = self.models_tr[name].transform([sample])
                step = 50
                found_k_class_n = self.k
                neigh_indexes = []
                while found_k_class_n>=0:
                    if self.common_neigh:
                        _, model_neig = self.gt_tree.query(self.counter.transform([sample]).toarray(), step)
                    else:
                        if hasattr(sample_trans, "toarray"):
                            _, model_neig = self.trees[name].query(sample_trans.toarray(), step)
                        else:
                            _, model_neig = self.trees[name].query(sample_trans, step)
                    for model_n_i in model_neig[0].tolist():
                        if name == 'lsi':
                            if self.true[model_n_i] != '35-49':
                                pass
                                # print 'GG'
                        if preds[model_ind] == self.true[model_n_i]:
                            neigh_indexes.append(model_n_i)
                            found_k_class_n -= 1
                    step *= 2
                    if step >= len(self.predictions[name]):
                        step = len(self.predictions[name])-1
                neigh_indexes = neigh_indexes[:self.k] 
                model_neig_pred = []
                neigh_true = []
                for model_n_i in neigh_indexes:
                    model_neig_pred.append(self.predictions[name][model_n_i])
                    neigh_true.append(self.true[model_n_i])
                lca[model_ind] = accuracy_score(neigh_true, model_neig_pred, normalize=True)
#                 print 'True Neigh: ' + str(neigh_true)
#                 print 'Predicted Neigh: ' + str(model_neig_pred)
                
                model_ind += 1
#             print 'LCA: %s' % str(['%0.2f' % (100*k) for k in lca])
#             print "Total Predicted: %s from model %s" % (str(preds[lca.index(max(lca))]), self.models.keys()[lca.index(max(lca))])
#             print '='*50
            return preds[lca.index(max(lca))]


    def predict_ola(self, sample):
        preds = []
        for name, model in self.models.iteritems():
            preds.append(model.predict([sample])[0])
#         print 'Preds: ' + str(preds)
        if len(set(preds))==1:
#             print 'Unanimous Decision: ' + str(preds[0])
#             print '='*50
            return preds[0]
        else:
            ola = [0 for pred in preds]
            model_ind = 0
            for name, model in self.models.iteritems():
#                 print 'Model: ' + name
                if self.common_neigh:
                    _, model_neig = self.gt_tree.query(self.counter.transform([sample]).toarray(), self.k)
                else:
                    sample_trans = self.models_tr[name].transform([sample])
                    if hasattr(sample_trans, "toarray"):
                        _, model_neig = self.trees[name].query(sample_trans.toarray(), self.k)
                    else:
                        _, model_neig = self.trees[name].query(sample_trans, self.k)
                model_neig_pred = []
                neigh_true = []
                for model_n_i in model_neig[0].tolist():
                    model_neig_pred.append(self.predictions[name][model_n_i])
                    neigh_true.append(self.true[model_n_i])
                ola[model_ind] = accuracy_score(neigh_true, model_neig_pred, normalize=True)
#                 print 'True Neigh: ' + str(neigh_true)
#                 print 'Predicted Neigh: ' + str(model_neig_pred)
#                 print 'OLA: %s' % str(['%0.2f' % (100*k) for k in ola])
                model_ind += 1
            
#             print "Total Predicted: %s from model %s" % (str(preds[ola.index(max(ola))]), self.models.keys()[ola.index(max(ola))])
#             print '='*50
            return preds[ola.index(max(ola))]

    def predict_kne(self, sample):
        preds = []
        for name, model in self.models.iteritems():
            preds.append(model.predict([sample])[0])
#         print 'Preds: ' + str(preds)
        if len(set(preds))==1:
#             print 'Unanimous Decision: ' + str(preds[0])
#             print '='*50
            return preds[0]
        else:
            k = self.k
            possible_experts = []
            neigh_radius = []
            ola_scores = []
            while k>0 :
                model_ind = 0
                # print k
                for name, model in self.models.iteritems():
#                     print 'Model: ' + name
                    if self.common_neigh:
                        _, model_neig = self.gt_tree.query(self.counter.transform([sample]).toarray(), k)
                    else:
                        sample_trans = self.models_tr[name].transform([sample])
                        if hasattr(sample_trans, "toarray"):
                            _, model_neig = self.trees[name].query(sample_trans.toarray(), k)
                        else:
                            _, model_neig = self.trees[name].query(sample_trans, k)
                    model_neig_pred = []
                    neigh_true = []
                    for model_n_i in model_neig[0].tolist():
                        model_neig_pred.append(self.predictions[name][model_n_i])
                        neigh_true.append(self.true[model_n_i])
#                     print 'True Neigh: ' + str(neigh_true)
#                     print 'Predicted Neigh: ' + str(model_neig_pred)
                    if k == self.k:
                        ola_scores.append(accuracy_score(neigh_true, model_neig_pred, normalize=True))
                    if neigh_true == model_neig_pred:
                        possible_experts.append(preds[model_ind])
                        neigh_radius.append(k)
                    model_ind += 1
                if not(possible_experts):
                    k -= 1
                else:
                    break
            if not(possible_experts):
#                 print 'No experts'
#                 print 'OLA_Scores: %s' % str(['%0.2f' % (100*k) for k in ola_scores])
#                 print preds[ola_scores.index(max(ola_scores))]
                return preds[ola_scores.index(max(ola_scores))]
            else:
#                 print 'Experts:'
#                 print possible_experts
#                 print neigh_radius
                return possible_experts[0]
            
     
    def predict_knu(self, sample):
        

        preds = []
        for name, model in self.models.iteritems():
            preds.append(model.predict([sample])[0])
        #print 'Preds: ' + str(preds)
        if len(set(preds))==1:
#             print 'Unanimous Decision: ' + str(preds[0])
#             print '='*50
            return preds[0]
        else:
            possible_experts = []
            neigh_radius = []
            ola_scores = []
            model_ind = 0
            for name, model in self.models.iteritems():
#                 print 'Model: ' + name
                if self.common_neigh:
                    _, model_neig = self.gt_tree.query(self.counter.transform([sample]).toarray(), self.k)
                else:
                    sample_trans = self.models_tr[name].transform([sample])
                    if hasattr(sample_trans, "toarray"):
                        _, model_neig = self.trees[name].query(sample_trans.toarray(), self.k)
                    else:
                        _, model_neig = self.trees[name].query(sample_trans, self.k)
                model_neig_pred = []
                neigh_true = []
                for model_n_i in model_neig[0].tolist():
                    model_neig_pred.append(self.predictions[name][model_n_i])
                    neigh_true.append(self.true[model_n_i])
                    if model_neig_pred[-1] == neigh_true[-1]:
                        possible_experts.append(preds[model_ind])
                ola_scores.append(accuracy_score(neigh_true, model_neig_pred, normalize=True))
#                 print 'True Neigh: ' + str(neigh_true)
#                 print 'Predicted Neigh: ' + str(model_neig_pred)
        if not(possible_experts):
#             print 'No experts'
#             print 'OLA_Scores: %s' % str(['%0.2f' % (100*k) for k in ola_scores])
#             print preds[ola_scores.index(max(ola_scores))]
            return preds[ola_scores.index(max(ola_scores))]
        else:
#             print 'Experts:'
#             print possible_experts
#             print most_common(possible_experts)
            return most_common(possible_experts)
                        
def most_common(lst):
    return max(set(lst), key=lst.count)
            


In [16]:
models11 = {}
models_tr11 = {}
for i, model in enumerate(trained_base_models[:]):
    models11[base_model_names[i]] = model
    models_tr11[base_model_names[i]] = trained_base_models[i].steps[0][1]        
LCA = Neigbors_DS(models11, models_tr11, k=3, scheme='OLA', common_neigh=False)
LCA.fit(X_meta, y_meta)
a = BaseClassifiers(base_predictions_cv[:], base_model_names[:], y_cv)
a.get_classification_report()
predict = LCA.predict(X_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Fitting time 14.78


AttributeError: Number of predictions of classifier 3grams is different                                      then the number of true labels. 65 != 131 .                                      

In [15]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split 
params = {
    #'common_neigh': [True, False],
    'scheme':['OLA', 'LCA', 'KNE', 'KNU']
}

models11 = {}
models_tr11 = {}
for i, model in enumerate(trained_base_models[:]):
    models11[base_model_names[i]] = model
    models_tr11[base_model_names[i]] = trained_base_models[i].steps[0][1]        
LCA = Neigbors_DS(models11, models_tr11, k=3, scheme='LCA', common_neigh=False)
#LCA.fit(X_train + X_meta, y_train + y_meta)

#X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.3, stratify=y, )
grid1 = GridSearchCV(LCA, param_grid =params, cv=5, verbose=1, n_jobs=3)
grid1.fit(X_meta, y_meta)
print(grid1.best_score_)
print(grid1.best_params_)
#pipe2 = grid1.best_estimator_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Fitting time 12.72
Fitting time 11.71


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without importing it

.............................................................

In [80]:
from sklearn.base import BaseEstimator, TransformerMixin
class Neigbors_Combinator(BaseEstimator, TransformerMixin):
    
    """GIving untrained Models and their transformers create a combinator of information:
       - Based on the count vectorizer predictions of a kNN classifier (fitted in X_train).
       - Based on the transformed kNN classifiers predictions (fitted in X_train)
       - Based on the samples predictions of the now trained models (fitted in X_train)
       - Combining the above predictions using weights (fitted in X_meta)
       
       Args:
           - models: dic of trained models in the form of: {"name": Model, :"name2":Model2...}
           - models_tr: dic of transformers of the models. Form like models.\
           - names: list of names for the models to be used
           - knn_params: dictionary containing params for fitting the knn classifiers.
                         Expects a dictionary like: {"general":{"num_folds":5, "n_jobs":-1},
                                                     "name":  parameter_grid_for_grid_search1,
                                                     "name2": parameter_grid_for_grid_search2,...}
            - scheme: for wegithing """
    
    def __init__(self, models, models_tr, names, knn_params, scheme='majority', weights=None):
        
        from combinations import Combinator
        from sklearn.feature_extraction.text import CountVectorizer
        
        self.counter = CountVectorizer()
        self.models = models
        self.models_tr = models_tr
        self.names = names
        self.knn_params = knn_params
        self.scheme = scheme
        self.weights = weights
        self.ind2names = {}
        for i, name in enumerate(names):
            self.ind2names[i] = name
        self.counter = CountVectorizer()
        self.gt_knn = None
        self.tr_knn = {}
        self.comb = Combinator(scheme=self.scheme)
            
        

    def fit(self, X_train, X_meta, y_train, y_meta, weights=None):
        
        from sklearn.grid_search import GridSearchCV
        from sklearn.neighbors import KNeighborsClassifier

        import random, time
        #print len(y_train), len(y_meta)
        if (y_train is None)  or (y_meta is None):
            raise ValueError('we need y labels to supervise-fit!')
        else:
            from sklearn.feature_extraction.text import CountVectorizer
        
            self.counter = CountVectorizer()
            parameters = {
                    'input': 'content',
                    'encoding': 'utf-8',
                    'decode_error': 'ignore',
                    'analyzer': 'word',
                    'stop_words': 'english',
                    # 'vocabulary':list(voc),
                    #'tokenizer': tokenization,
                    #'tokenizer': _twokenize.tokenizeRawTweetText,  # self.tokenization,
                    #'tokenizer': lambda text: _twokenize.tokenizeRawTweetText(nonan.sub(po_re.sub('', text))),
                    'max_df': 1.0,
                    'min_df': 1,
                    'max_features':None
                }
            
            # Fit the Meta-Classifier
            predictions_meta = []
            # Fitting the ground truth neighbors
            t0 = time.time()
            self.counter.set_params(**parameters)
            X_train_truth = self.counter.fit_transform(X_train)
            #print X_train_truth.toarray().shape
            grid_ = GridSearchCV(KNeighborsClassifier(), param_grid=self.knn_params['true'],
                                    n_jobs=self.knn_params['general']['n_jobs'], 
                                     cv=self.knn_params['general']['cv'], refit=True, 
                                      verbose=self.knn_params['general']['verbose'])
            grid_.fit(X_train_truth, y_train)
            self.gt_knn = grid_.best_estimator_
            # Add the gt predictions
            #print self.counter.transform(X_meta).toarray().shape
            predictions_meta.append(self.gt_knn.predict(self.counter.transform(X_meta)))
            # Fit the transformation Classifiers
            for name in self.names:
                X_train_tr = self.models_tr[name].transform(X_train)
                grid_ = GridSearchCV(KNeighborsClassifier(), param_grid=self.knn_params[name],
                                    n_jobs=self.knn_params['general']['n_jobs'], 
                                     cv=self.knn_params['general']['cv'], refit=True, 
                                      verbose=self.knn_params['general']['verbose'])
                grid_.fit(X_train_tr, y_train)
                self.tr_knn[name] = grid_.best_estimator_
                # Add the transformed knn predictions
                predictions_meta.append(self.tr_knn[name].predict(self.models_tr[name].transform(X_meta)))
                # Add the per sample predictions
                predictions_meta.append(self.models[name].predict(X_meta))
            #print len(predictions_meta)
            #print predictions_meta
            self.comb.fit(predictions_meta, y_meta)
            print "Best Weights Found:"
            print self.comb.weights
            return
        
        
    def transform(self, X, y):

        import numpy
        
        predictions = []
        X_truth = self.counter.transform(X)   
        predictions.append(self.gt_knn.predict(X_truth))
        for name in self.names:
            #print name
            # Add the transformed knn predictions
            predictions.append(self.tr_knn[name].predict(self.models_tr[name].transform(X)))
            # Add the per sample predictions
            predictions.append(self.models[name].predict(X))
            #print len(predictions)
        #print len(predictions)
        #for pred in predictions:
        #    print len(pred)
        final_predict =self.comb.predict(predictions)
        names = ['Ground_Truth', '3grams-Neighbors', '3grams-Pred',
                     'SOAC-Neigbors', 'SOAC-Pred', 'LSI-Neighbors', 'LSI-Pred', 'Comb']
        for i in xrange(len(y)):
            print 'True: ' + str(y[i])
            for  j, pred in enumerate(predictions):
                print '%s: %s' % (names[j], pred[i])
            print "Comb: %s" % final_predict[i]
            print '*'*100
        return final_predict

    def predict(self, X):
        return self.transform(X)

    def score(self, X, y_true):

        from sklearn.metrics import accuracy_score

        y_pred = self.predict(X)
        return accuracy_score(y_true, y_pred, normalize=True)

models11 = {}
models_tr11 = {}
tr_params = {'n_neighbors':[i for i in xrange(1,20)], 'weights':['uniform', 'distance']}
knn_params = {"general":{"n_jobs":-1, "cv":5, "verbose":1}}
knn_params['true'] = tr_params
names11 = []
for i, model in enumerate(trained_base_models):
    names11.append(base_model_names[i])
    models11[names11[-1]] = model
    models_tr11[names11[-1]] = trained_base_models[i].steps[0][1]
    knn_params[names11[-1]] = tr_params
    #print base_model_names[i]
    #print trained_base_models[i].steps[0][1]
aa = Neigbors_Combinator(models11, models_tr11, names11, knn_params, scheme='accuracy')            
aa.fit(X_train, X_meta, y_train, y_meta)
predict = aa.predict(X_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

AttributeError: lower not found

In [ ]:
for i in xrange(len(y_meta)):
    print "True Label: %s" % y_meta[i]
    prediction_string = ''
    for name, model in zip(base_model_names, pred_cv):
        prediction_string += ' %s:%s |' % (name, g.lab.inverse_transform(g.transformation(model).argmax(axis=1))[0])
    prediction_string += ' %s:%s' % ('Bernoulli', predict[i])
    print prediction_string[1:]

In [159]:
g = combinations.Combinator(scheme='majority')
g.fit(base_predictions_meta, y_meta)
pred_meta = g.lab.transform(base_predictions_meta)
pred_meta = g.ohe.transform(pred_meta.reshape(-1, 1)).todense().reshape(len(base_predictions_meta[0]), -1)
pred_cv = g.lab.transform(base_predictions_cv)
pred_cv = g.ohe.transform(pred_cv.reshape(-1, 1)).todense().reshape(len(base_predictions_cv[0]), -1)
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
gg =MultinomialNB()
gg.fit(pred_meta, y_meta)
gg.predict(pred_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Accuracy : 0.292307692308
Confusion matrix :
 [[ 0  0  3  1  0]
 [ 0  0 20  1  0]
 [ 0  7 18  1  1]
 [ 0  4  7  1  0]
 [ 0  0  0  1  0]]
Classification report :
              precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         4
      25-34       0.00      0.00      0.00        21
      35-49       0.38      0.67      0.48        27
      50-64       0.20      0.08      0.12        12
      65-xx       0.00      0.00      0.00         1

avg / total       0.19      0.29      0.22        65



In [60]:
import EnsembleDiversityTests
reload(EnsembleDiversityTests)
from EnsembleDiversityTests import BaseClassifiers

a = BaseClassifiers(base_predictions_meta, base_model_names, y_meta)
g = a.get_per_class_accuracy()
a1 = g.T['3grams']

Per Class Accuracy of the models
        18-24  25-34       35-49  50-64  65-xx
3grams      0     25   57.142857      0      0
soac       50     10   67.857143      0      0
lsi         0      0  100.000000      0      0


In [71]:
from sklearn.base import BaseEstimator, TransformerMixin
class Neigbors_Combinator(BaseEstimator, TransformerMixin):
    
    """GIving untrained Models and their transformers create a combinator of information:
       - Based on the count vectorizer predictions of a kNN classifier (fitted in X_train).
       - Based on the transformed kNN classifiers predictions (fitted in X_train)
       - Based on the samples predictions of the now trained models (fitted in X_train)
       - Combining the above predictions using weights (fitted in X_meta) based ond LCA
       
       Args:
           - models: dic of trained models in the form of: {"name": Model, :"name2":Model2...}
           - models_tr: dic of transformers of the models. Form like models.\
           - names: list of names for the models to be used
           - knn_params: dictionary containing params for fitting the knn classifiers.
                         Expects a dictionary like: {"general":{"num_folds":5, "n_jobs":-1},
                                                     "name":  parameter_grid_for_grid_search1,
                                                     "name2": parameter_grid_for_grid_search2,...}
            - scheme: for weigting """
    
    def __init__(self, models, models_tr, names, knn_params, scheme='majority', weights=None):
        
        from combinations import Combinator
        from sklearn.feature_extraction.text import CountVectorizer
        
        self.counter = CountVectorizer()
        self.models = models
        self.models_tr = models_tr
        self.names = names
        self.knn_params = knn_params
        self.scheme = scheme
        self.weights = weights
        self.ind2names = {}
        for i, name in enumerate(names):
            self.ind2names[i] = name
        self.counter = CountVectorizer()
        self.gt_knn = None
        self.tr_knn = {}
        self.comb = Combinator(scheme=self.scheme)
            
        

    def fit(self, X_train, X_meta, y_train, y_meta, weights=None):
        
        from sklearn.grid_search import GridSearchCV
        from sklearn.neighbors import KNeighborsClassifier
from sklearn.e        
        from EnsembleDiversityTests import BaseClassifiers

        import random, time
        #print len(y_train), len(y_meta)
        if (y_train is None)  or (y_meta is None):
            raise ValueError('we need y labels to supervise-fit!')
        else:
            parameters = {
                    'input': 'content',
                    'encoding': 'utf-8',
                    'decode_error': 'ignore',
                    'analyzer': 'word',
                    'stop_words': 'english',
                    # 'vocabulary':list(voc),
                    #'tokenizer': tokenization,
                    #'tokenizer': _twokenize.tokenizeRawTweetText,  # self.tokenization,
                    #'tokenizer': lambda text: _twokenize.tokenizeRawTweetText(nonan.sub(po_re.sub('', text))),
                    'max_df': 1.0,
                    'min_df': 1,
                    'max_features':None
                }
            
            # Fit the Meta-Classifier
            predictions_meta = []
            # Fitting the ground truth neighbors
            t0 = time.time()
            self.counter.set_params(**parameters)
            X_train_truth = self.counter.fit_transform(X_train)
            #print X_train_truth.toarray().shape
            grid_ = GridSearchCV(KNeighborsClassifier(), param_grid=self.knn_params['true'],
                                    n_jobs=self.knn_params['general']['n_jobs'], 
                                     cv=self.knn_params['general']['cv'], refit=True, 
                                      verbose=self.knn_params['general']['verbose'])
            grid_.fit(X_train_truth, y_train)
            self.gt_knn = grid_.best_estimator_
            # Add the gt predictions
            #print self.counter.transform(X_meta).toarray().shape
            predictions_meta.append(self.gt_knn.predict(self.counter.transform(X_meta)))
            # Fit the transformation Classifiers
            for name in self.names:
                X_train_tr = self.models_tr[name].transform(X_train)
                grid_ = GridSearchCV(KNeighborsClassifier(), param_grid=self.knn_params[name],
                                    n_jobs=self.knn_params['general']['n_jobs'], 
                                     cv=self.knn_params['general']['cv'], refit=True, 
                                      verbose=self.knn_params['general']['verbose'])
                grid_.fit(X_train_tr, y_train)
                self.tr_knn[name] = grid_.best_estimator_
                # Add the transformed knn predictions
                predictions_meta.append(self.tr_knn[name].predict(self.models_tr[name].transform(X_meta)))
                # Add the per sample predictions
                predictions_meta.append(self.models[name].predict(X_meta))
            #print len(predictions_meta)
            #print predictions_meta
            
            self.comb.fit(predictions_meta, y_meta)
            print "Best Weights Found:"
            print self.comb.weights
            return
        
        
    def transform(self, X, y):

        import numpy
        
        predictions = []
        X_truth = self.counter.transform(X)   
        predictions.append(self.gt_knn.predict(X_truth))
        for name in self.names:
            #print name
            # Add the transformed knn predictions
            predictions.append(self.tr_knn[name].predict(self.models_tr[name].transform(X)))
            # Add the per sample predictions
            predictions.append(self.models[name].predict(X))
            #print len(predictions)
        #print len(predictions)
        #for pred in predictions:
        #    print len(pred)
        final_predict =self.comb.predict(predictions)
        names = ['Ground_Truth', '3grams-Neighbors', '3grams-Pred',
                     'SOAC-Neigbors', 'SOAC-Pred', 'LSI-Neighbors', 'LSI-Pred', 'Comb']
        for i in xrange(len(y)):
            print 'True: ' + str(y[i])
            for  j, pred in enumerate(predictions):
                print '%s: %s' % (names[j], pred[i])
            print "Comb: %s" % final_predict[i]
            print '*'*100
        return final_predict

    def predict(self, X, y):
        return self.transform(X)

    def score(self, X, y_true):

        from sklearn.metrics import accuracy_score

        y_pred = self.predict(X)
        return accuracy_score(y_true, y_pred, normalize=True)

models11 = {}
models_tr11 = {}
tr_params = {'n_neighbors':[i for i in xrange(1,20)], 'weights':['uniform', 'distance']}
knn_params = {"general":{"n_jobs":-1, "cv":5, "verbose":1}}
knn_params['true'] = tr_params
names11 = []
for i, model in enumerate(trained_base_models):
    names11.append(base_model_names[i])
    models11[names11[-1]] = model
    models_tr11[names11[-1]] = trained_base_models[i].steps[0][1]
    knn_params[names11[-1]] = tr_params
    #print base_model_names[i]
    #print trained_base_models[i].steps[0][1]
aa = Neigbors_Combinator(models11, models_tr11, names11, knn_params, scheme='accuracy')            
aa.fit(X_train, X_meta, y_train, y_meta)
predict = aa.predict(X_cv, y_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Fitting 5 folds for each of 38 candidates, totalling 190 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 190 out of 190 | elapsed:    3.7s finished


Fitting 5 folds for each of 38 candidates, totalling 190 fits
Fitting 5 folds for each of 38 candidates, totalling 190 fits


[Parallel(n_jobs=-1)]: Done 190 out of 190 | elapsed:    0.4s finished


Fitting 5 folds for each of 38 candidates, totalling 190 fits


[Parallel(n_jobs=-1)]: Done 190 out of 190 | elapsed:    0.3s finished


Best Weights Found:
[21, 17, 21, 26, 23, 22, 28]


[Parallel(n_jobs=-1)]: Done 190 out of 190 | elapsed:    0.3s finished


TypeError: transform() takes exactly 3 arguments (2 given)

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class SubSpaceEnsemble3(BaseEstimator, TransformerMixin):
    
    """ Utilizing the neighborhood in all representations and also ground truth model.
        Implementing a weighted voting scheme."""

    def __init__(self, models, k=3, weights= [2,1,3,0.7]):
        from sklearn.feature_extraction.text import CountVectorizer
        
        if (not models):
            raise AttributeError('Models expexts a dictonary of models \
              containg the predictions of y_true for each classifier.\ ')
        else:
            self.models = models
            # self.cv_scores = cv_scores
            self.k = k
            self.weights = weights
            self.ind2names = {}
            for i, name in enumerate(models.keys()):
                self.ind2names[i] = name
            self.counter = CountVectorizer()
            self.representations = []
            self.meta = None
            self.predictions = []
            self.true = []
            self.doc_terms = None
            self.tree = None
            self.experts = []
        

    def fit(self, X_cv, y_true=None, weights=None):
        
        from sklearn.neighbors import BallTree
        import random

        if y_true is None:
            raise ValueError('we need y labels to supervise-fit!')
        else:
            parameters = {
                    'input': 'content',
                    'encoding': 'utf-8',
                    'decode_error': 'ignore',
                    'analyzer': 'word',
                    'stop_words': 'english',
                    # 'vocabulary':list(voc),
                    #'tokenizer': tokenization,
                    #'tokenizer': _twokenize.tokenizeRawTweetText,  # self.tokenization,
                    #'tokenizer': lambda text: _twokenize.tokenizeRawTweetText(nonan.sub(po_re.sub('', text))),
                    'max_df': 1.0,
                    'min_df': 1,
                    'max_features':None
                }
            t0 = time.time()
            self.counter.set_params(**parameters)
            self.doc_terms = self.counter.fit_transform(X_cv).toarray()
            self.tree = BallTree(self.doc_terms, leaf_size=20)
            predictions = []
            for name, model in self.models.iteritems():
                predictions.append(model.predict(X_cv))
                #print len(predictions[-1])
                transf = model.steps[0][1].transform(X_cv)
                if hasattr(transf, "toarray"):
                    #print 'Exei'
                    self.representations.append(transf.toarray())
                else:
                    self.representations.append(transf)
            self.predictions = predictions
            self.true = y_true
            count = 0
            #print self.expert_scores
            #print self.experts
            print('Fit took: %0.3f seconds') % (time.time()-t0)
            return self

    def predict(self, X):
        # print "PRedict"
        # print X.shape
        X_transformed = self.counter.transform(X).toarray()
        #print type((X_transformed)[0])
        #print X_transformed.shape
        #return 0
        y_pred = []
        t0 = time.time()
        for i in range(0, X_transformed.shape[0]):
            #print X_transformed[i,:].shape
            dist, neigbors_indexes = self.tree.query(X_transformed[i,:].reshape(1,-1), self.k)  
#             print 'Sample ' + y_real[i]
            #print dist
            #print type(dist)
            #print neigbors_indexes[0]
            #print dist
            #best_model_ind = self.expert_decision(neigbors_indexes[0])
            #pass
            y_pred.append(self.expert_decision(neigbors_indexes[0],  dist, X[i]))
            
            #y_pred.append(self.models[self.ind2names[best_model_ind]].predict([X[i]])[0])
        #print y_pred
        print('Predict took: %0.3f seconds') % (time.time()-t0)
        return y_pred

    def score(self, X, y, sample_weight=None):

        from sklearn.metrics import accuracy_score
        return accuracy_score(y, self.predict(X), normalize=True)
        #return self.svc.score(self.transform_to_y(X), y, sample_weight)


    def expert_decision(self, neigbors_indexes, dist, x_sample):

        from sklearn.metrics import accuracy_score
        from collections import Counter
        from sklearn.neighbors import BallTree
        
        models_pred = []
        models_neig_pred = []
        acc = []
        t0 = time.time()
        neigbors_true = [self.true[n_i] for n_i in neigbors_indexes]
        #print('Neighbors per sample: %0.4f seconds') % (time.time()-t0)
#         print 'True'
#         print neigbors_true
        sample_predictions = []
        total_pred = []
        weights = {}
        weights['true'] = self.weights[2]
        weights['models_n'] = []
        weights['models'] = []
        for model_i in xrange(len(self.models.values())):
            ModelTree = BallTree(self.representations[model_i])
            temp_trans = self.models[self.ind2names[model_i]].steps[0][1].transform([x_sample])
            if hasattr(temp_trans, 'toarray'):
                temp_trans = temp_trans.toarray()
            _, model_neig = ModelTree.query(temp_trans, self.k)
            model_neig_pred = []
            for model_n_i in model_neig[0].tolist():
                model_neig_pred.append(self.predictions[model_i][model_n_i])
            models_neig_pred.append(model_neig_pred)
            model_pred = []
            for n_i in neigbors_indexes:
                model_pred.append(self.predictions[model_i][n_i])
            models_pred.append(model_pred)
            acc.append(accuracy_score(neigbors_true, model_neig_pred, normalize=True))
            if acc[-1] >self.weights[3]:
                # Adding neighbors predictions
                weights['models_n'].append(int(self.weights[1]/float((1-acc[-1])+0.01)))
                total_pred.extend([pred for j in xrange(weights['models_n'][-1]) for pred in model_pred])
                #print('Predicting Neighbors per sample: %0.4f seconds') % (time.time()-t0)
                # Adding sample prediction
                sample_predictions.append(self.models[self.ind2names[model_i]].predict(x_sample)[0])
                weights['models'].append(int(self.weights[0]/float((1-acc[-1])+0.01))) 
                total_pred.extend([sample_predictions[-1] for j in xrange(weights['models'][-1])])
                total_pred.extend([pred for j in xrange(weights['models'][-1]) for pred in model_neig_pred])
            #print len(x_sample)
            #print self.ind2names[model_i]
            
#                 print 'Model: ' + self.ind2names[model_i] + ' Accuracy: ' + str(accuracy_score(neigbors_true, model_neig_pred, normalize=True))
#                 print 'Predictions'
#                 print model_pred
#                 print 'Representations'
#                 print model_neig_pred
#                 print 'Sample prediction: ' + str(sample_predictions[-1])
        total_pred.extend([n for i, n in enumerate(neigbors_true) for j in xrange(int(weights['true']*(self.k-i)))])
        #print('creating votes: %0.4f seconds') % (time.time()-t0)
        data = Counter(total_pred)
        #data = Counter([k for pred in models_pred for k in pred])
#         print data
#         best_model_ind = acc.index(max(acc))
#         print 'Total pred: ' + str(data.most_common(1)[0][0])
#         print '='*50
#         #print len(total_pred)
        #return best_model_ind
        return data.most_common(1)[0][0]

In [22]:
space2 = SubSpaceEnsemble3(models_for_space, k=3, weights= [2,1,3,-1])
#2,1,3,0.7
space2.fit(X_meta, y_meta)
predict = space2.predict(X_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Fit took: 16.118 seconds
Predict took: 1215.613 seconds
Accuracy : 0.369230769231
Confusion matrix :
 [[ 0  2  2  0  0]
 [ 0  6 15  0  0]
 [ 1  7 18  1  0]
 [ 0  3  9  0  0]
 [ 0  0  1  0  0]]
Classification report :
              precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         4
      25-34       0.33      0.29      0.31        21
      35-49       0.40      0.67      0.50        27
      50-64       0.00      0.00      0.00        12
      65-xx       0.00      0.00      0.00         1

avg / total       0.27      0.37      0.31        65



In [18]:
from sklearn.base import BaseEstimator, TransformerMixin

class SubSpaceEnsemble3_w(BaseEstimator, TransformerMixin):
    
    """ Utilizing the neighborhood in all representations and also ground truth model.
        Implementing a weighted voting scheme. Finding Optimal weights also!"""

    def __init__(self, models, k=3, scheme = 'weights', weights=[6,3,2,0.7]):
        from sklearn.feature_extraction.text import CountVectorizer
        
        if (not models) :
            raise AttributeError('Models expexts a dictonary of models \
              containg the predictions of y_true for each classifier.\
              cv_score expects a list len(models.keys()) with the\
              cross validation scores of each model')
        else:
            self.models = models
            self.k = k
            self.scheme = scheme
            self.weights = None
            self.ind2names = {}
            for i, name in enumerate(models.keys()):
                self.ind2names[i] = name
            self.counter = CountVectorizer()
            self.representations = []
            self.meta = None
            self.predictions = []
            self.true = []
            self.doc_terms = None
            self.tree = None
            self.experts = []
            if self.scheme == 'weights':
                if type(self.weights) in (numpy.array, numpy.ndarray):
                    pass  # It is from the optimization part
                else:
                    if not(self.weights):
                        print "Need weights for this scheme!"
                self.weights = weights
                weights_string = " %.2f |" * len(self.weights) % tuple(self.weights)
                # print "Using given weights: | %s" % weights_string
            elif self.scheme == 'optimal':
                # print "Will find the weights after fitting"
                pass
            else:
                self.weights = [6,3,2,0.7]
        

    def fit(self, X, X_cv, y = None, y_true=None, weights=None):
        
        from sklearn.neighbors import BallTree
        import random

        if y_true is None:
            raise ValueError('we need y labels to supervise-fit!')
        else:
            parameters = {
                    'input': 'content',
                    'encoding': 'utf-8',
                    'decode_error': 'ignore',
                    'analyzer': 'word',
                    'stop_words': 'english',
                    # 'vocabulary':list(voc),
                    #'tokenizer': tokenization,
                    #'tokenizer': _twokenize.tokenizeRawTweetText,  # self.tokenization,
                    #'tokenizer': lambda text: _twokenize.tokenizeRawTweetText(nonan.sub(po_re.sub('', text))),
                    'max_df': 1.0,
                    'min_df': 1,
                    'max_features':None
                }
            t0 = time.time()
            self.counter.set_params(**parameters)
            self.doc_terms = self.counter.fit_transform(X + X_cv).toarray()
            self.tree = BallTree(self.doc_terms, leaf_size=20)
            predictions = []
            for name, model in self.models.iteritems():
                predictions.append(model.predict(X + X_cv))
                #print len(predictions[-1])
                transf = model.steps[0][1].transform(X + X_cv)
                if hasattr(transf, "toarray"):
                    #print 'Exei'
                    self.representations.append(transf.toarray())
                else:
                    self.representations.append(transf)
            self.predictions = predictions
            self.true = y + y_true
            count = 0
            #print self.expert_scores
            #print self.experts
            if self.scheme=='optimal':
                self.find_weights(X, X_cv, y, y_cv)
                weights_string = " %.2f |" * len(self.weights) % tuple(self.weights)
            print('Fit took: %0.3f seconds') % (time.time()-t0)
            return self
        
    def find_weights(self, X, X_cv, y, y_cv):

        import numpy

        from scipy.optimize import minimize

        w = [6,3,2,0.7]
        bnds = tuple([(0, None) for i in xrange(len(w))])
        a = minimize(self.f, w, args=(SubSpaceEnsemble3_w, X, X_cv, y, y_cv), method='L-BFGS-B', bounds=bnds)
        self.weights = list(a.x)
        return
    
    def find_weigths2(self, X, X_cv, y):
        
        N_rand = 100
        N_rand1 = 50
        poss_w = []
        acc_ = []
        pred = []
        for i in xrange(N_rand1):
            tmp_w = [2,1,3,6]
            tmp_w[0] = round(random.random(), 3)
            tmp_w[1] = round(1 - tmp_w[0], 3)
            tmp_w[2] = round(random.uniform(0.2, 0.8), 3)
            # tmp_w[3] = random.randint(1,10)
            poss_w.append(tmp_w)
            pred = self.find_weights(X_cv, tmp_w)
            acc = accuracy_score(self.true, pred)
            # print('Accuracy : {}'.format(acc))
            acc_.append(acc)
        print('First search took: %0.3f seconds') % (time.time() - t0)
        tmp_w = poss_w[acc_.index(max(acc_))]
        poss_w = []
        acc_ = []
        for i in xrange(self.N_rand  -N_rand1):
            tmp_w2 = tmp_w
            tmp_w2[0] = round(random.uniform(tmp_w[0] - 0.1, tmp_w[0] + 0.1), 3)
            tmp_w2[1] = round(1 - tmp_w2[0], 3)
            tmp_w2[2] = round(random.uniform(tmp_w[2] - 0.1, tmp_w[1] + 0.1), 3)
            poss_w.append(tmp_w2)
            pred = self.find_weights(X_cv, tmp_w2)
            acc = accuracy_score(self.true, pred)
            # print('Accuracy : {}'.format(acc))
            acc_.append(acc)
        self.weights = poss_w[acc_.index(max(acc_))]
        self.k = self.weights[3]

    def f(self, w, SubSpaceEnsemble3_w, X, X_cv, y, y_cv):
        gg = SubSpaceEnsemble3_w(self.models, self.k, scheme= 'weights', weights= w)
        gg.fit(X, X_cv, y, y_cv)
        score = 1 - gg.score(x, y)
        # print 'Weights'
        # print w
        # print 'Score: ' + str(score)
        return score

    def predict(self, X):
        
        # print "PRedict"
        # print X.shape
        X_transformed = self.counter.transform(X).toarray()
        #print type((X_transformed)[0])
        #print X_transformed.shape
        #return 0
        y_pred = []
        t0 = time.time()
        for i in range(0, X_transformed.shape[0]):
            #print X_transformed[i,:].shape
            dist, neigbors_indexes = self.tree.query(X_transformed[i,:].reshape(1,-1), self.k)  
            # print 'Sample ' + y_real[i]
            #print neigbors_indexes[0]
            #print dist
            #best_model_ind = self.expert_decision(neigbors_indexes[0])
            y_pred.append(self.expert_decision(neigbors_indexes[0],  X[i]))
            
            #y_pred.append(self.models[self.ind2names[best_model_ind]].predict([X[i]])[0])
        #print y_pred
        print('Predict took: %0.3f seconds') % (time.time()-t0)
        return y_pred

    def score(self, X, y, sample_weight=None):

        from sklearn.metrics import accuracy_score
        return accuracy_score(y, self.predict(X), normalize=True)
        #return self.svc.score(self.transform_to_y(X), y, sample_weight)


    def expert_decision(self, neigbors_indexes, x_sample):

        from sklearn.metrics import accuracy_score
        from collections import Counter
        from sklearn.neighbors import BallTree
        
        models_pred = []
        models_neig_pred = []
        acc = []
        t0 = time.time()
        neigbors_true = [self.true[n_i] for n_i in neigbors_indexes]
        #print('Neighbors per sample: %0.4f seconds') % (time.time()-t0)
#         print 'True'
#         print neigbors_true
        sample_predictions = []
        total_pred = []
        weights = {}
        weights['true'] = self.weights[2]
        weights['models_n'] = []
        weights['models'] = []
        for model_i in xrange(len(self.models.values())):
            ModelTree = BallTree(self.representations[model_i])
            temp_trans = self.models[self.ind2names[model_i]].steps[0][1].transform([x_sample])
            if hasattr(temp_trans, 'toarray'):
                temp_trans = temp_trans.toarray()
            _, model_neig = ModelTree.query(temp_trans, self.k)
            model_neig_pred = []
            for model_n_i in model_neig[0].tolist():
                model_neig_pred.append(self.predictions[model_i][model_n_i])
            models_neig_pred.append(model_neig_pred)
            model_pred = []
            for n_i in neigbors_indexes:
                model_pred.append(self.predictions[model_i][n_i])
            models_pred.append(model_pred)
            acc.append(accuracy_score(neigbors_true, model_pred, normalize=True))
            if acc[-1] >self.weights[3]:
                # Adding neighbors predictions
                weights['models_n'].append(int(self.weights[1]/float((1-acc[-1])+0.01)))
                total_pred.extend([pred for j in xrange(weights['models_n'][-1]) for pred in model_pred])
                #print('Predicting Neighbors per sample: %0.4f seconds') % (time.time()-t0)
                # Adding sample prediction
                sample_predictions.append(self.models[self.ind2names[model_i]].predict(x_sample)[0])
                weights['models'].append(int(self.weights[0]/float((1-acc[-1])+0.01))) 
                total_pred.extend([sample_predictions[-1] for j in xrange(weights['models'][-1])])
                total_pred.extend([pred for j in xrange(weights['models'][-1]) for pred in model_neig_pred])
            #print len(x_sample)
            #print self.ind2names[model_i]
            
#                 print 'Model: ' + self.ind2names[model_i] + ' Accuracy: ' + str(accuracy_score(neigbors_true, model_pred, normalize=True))
#                 print 'Predictions'
#                 print model_pred
#                 print 'Representations'
#                 print model_neig_pred
#                 print 'Sample prediction: ' + str(sample_predictions[-1])
        total_pred.extend([n for j in xrange(int(weights['true'])) for n in neigbors_true])
        #print('creating votes: %0.4f seconds') % (time.time()-t0)
        data = Counter(total_pred)
        #data = Counter([k for pred in models_pred for k in pred])
#         print data
#         best_model_ind = acc.index(max(acc))
#         print 'Total pred: ' + str(data.most_common(1)[0][0])
        #print len(total_pred)
        #return best_model_ind
        return data.most_common(1)[0][0]

In [19]:
space_k = SubSpaceEnsemble3_w(models_for_space, k=3, scheme='optimal', weights=[2,1,3,0.5])
space_k.fit(X,X_meta, y,y_meta)
predict = space_k.predict(X_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Need weights for this scheme!
Fit took: 100.444 seconds


KeyboardInterrupt: 

In [146]:
#import EnsembleDiversityTests
reload(EnsembleDiversityTests)
from EnsembleDiversityTests import DiversityTests, BaseClassifiers
#gg = DiversityTests(predictions[:-1], print_names[:-1], predictions[-1])
gg = DiversityTests(base_predictions_cv, base_model_names, y_cv)
#gg.print_report()
gg1 = BaseClassifiers(base_predictions_cv, base_model_names, y_cv, True)
#gg1.get_comparison_report()

In [147]:
gg1.get_per_class_accuracy()

Per Class Accuracy of the models!
        18-24      25-34      35-49      50-64  65-xx
3grams      0  47.619048  74.074074   0.000000      0
soac        0  52.380952  40.740741  16.666667      0
lsi       100   0.000000   0.000000   0.000000      0


In [109]:
from sklearn import tree
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
tree1 = tree.DecisionTreeClassifier()
from sklearn.preprocessing import LabelEncoder
lab = LabelEncoder()
y_meta_tr = lab.fit_transform(y_meta)
X_meta_trees = []
for i in xrange(len(y_meta)):
    tmp = []
    for j in xrange(len(base_predictions_meta)):
        tmp.append(base_predictions_meta[j][i])
    X_meta_trees.append(tmp)
#
X_meta_trees = numpy.array(X_meta_trees, dtype='|S5')


X_meta_trees = lab.transform(X_meta_trees.reshape(-1,1)).reshape(len(y_meta), len(base_predictions_meta))
#params = {'criterion': ['gini', 'entropy'], 'min_samples_split': list(xrange(1,10)), 
#          'max_depth': [None]+list(xrange(1,5)), 'min_samples_leaf': list(xrange(1,4)) }
params = {'criterion': ['gini', 'entropy'], 'n_estimators': [100, 1000, 10000],
          'min_samples_split': list(xrange(1,10)), 'max_depth': [None]+list(xrange(1,5)), 
          'min_samples_leaf': list(xrange(1,4)) }

grid_ = GridSearchCV(RandomForestClassifier(), param_grid = params, n_jobs= -1, cv=10, refit=True, verbose=1)
#grid_ = GridSearchCV(tree1, param_grid = params, n_jobs= -1, cv=10, refit=True, verbose=1)
grid_.fit(numpy.array(X_meta_trees), y_meta_tr)
tree1 = grid_.best_estimator_
#tree1.fit(numpy.array(X_meta_trees), y_meta_tr)

y_cv_tr = lab.transform(y_cv)
X_cv_trees = []
for i in xrange(len(y_cv)):
    tmp = []
    for j in xrange(len(base_predictions_cv)):
        tmp.append(base_predictions_cv[j][i])
    X_cv_trees.append(tmp)
#
X_cv_trees = numpy.array(X_cv_trees, dtype='|S5')

X_cv_trees = lab.transform(X_cv_trees.reshape(-1,1)).reshape(len(y_cv), len(base_predictions_cv))
predict = lab.inverse_transform(tree1.predict(X_cv_trees))
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Fitting 10 folds for each of 810 candidates, totalling 8100 fits


[Parallel(n_jobs=-1)]: Done  55 tasks      | elapsed:   58.0s
[Parallel(n_jobs=-1)]: Done 205 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 455 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 805 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done 1255 tasks      | elapsed: 23.1min
[Parallel(n_jobs=-1)]: Done 1805 tasks      | elapsed: 32.9min
[Parallel(n_jobs=-1)]: Done 2455 tasks      | elapsed: 44.9min
[Parallel(n_jobs=-1)]: Done 3205 tasks      | elapsed: 58.6min
[Parallel(n_jobs=-1)]: Done 4055 tasks      | elapsed: 74.0min
[Parallel(n_jobs=-1)]: Done 5005 tasks      | elapsed: 91.6min
[Parallel(n_jobs=-1)]: Done 6055 tasks      | elapsed: 110.8min
[Parallel(n_jobs=-1)]: Done 7205 tasks      | elapsed: 131.8min
[Parallel(n_jobs=-1)]: Done 8100 out of 8100 | elapsed: 148.5min finished


Accuracy : 0.338461538462
Confusion matrix :
 [[ 0  1  3  0  0]
 [ 0  0 21  0  0]
 [ 0  5 22  0  0]
 [ 0  5  7  0  0]
 [ 0  0  1  0  0]]
Classification report :
              precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         4
      25-34       0.00      0.00      0.00        21
      35-49       0.41      0.81      0.54        27
      50-64       0.00      0.00      0.00        12
      65-xx       0.00      0.00      0.00         1

avg / total       0.17      0.34      0.23        65



In [203]:
from sklearn.base import BaseEstimator, TransformerMixin
from pan.features import tokenization2

class Freq(BaseEstimator, TransformerMixin):
    
    """ Find the frequency of votes corresponding to specific classes.
        It does not learn anything new. Just counting on the evaluation
        dataset.
        
        Args:
    """
    
    
    def __init__(self):
        
        from sklearn.preprocessing import LabelEncoder, OneHotEncoder
        from sklearn.naive_bayes import BernoulliNB
        
        self.rule_book = None
        self.rule_index = {}
        self.transformed = None
        self.labels = None
        self.num_models = None
        self.num_labels = None
        self.lab = LabelEncoder()
        self.ohe = OneHotEncoder()
        self.Bernoulli = BernoulliNB()
        
        
        
    def fit(self, X, y):
        """Expects a list of string inputs as X and a list of lists containing
           the prediciton of each model for y"""
        
        import numpy
        
        X_tr, y_tr = self.fit_encoders(X, y)
        self.transformed = X_tr
        tmp = numpy.zeros([1, self.num_labels])
        j = 0
        self.rule_book = numpy.copy(tmp)
        for i, x in enumerate(X_tr):
            x = str(x)
            #print x
            #print self.rule_book
            if x in self.rule_index.keys():
                #print y_tr[i].argmax(axis=1)
                #print self.rule_book[self.rule_index[x], y_tr.argmax(axis=1)].shape
                self.rule_book[self.rule_index[x], y_tr[i].argmax(axis=1)] += 1
                #print self.rule_book[self.rule_index[x], :]
            else:
                self.rule_index[x] = j
                self.rule_book = numpy.vstack((self.rule_book, tmp))
                #self.rule_book = numpy.delete(self.rule_book, 0, 0)
                self.rule_book[self.rule_index[x], y_tr[i].argmax(axis=1)] += 1
                j += 1
        self.num_rules = len(list(self.rule_index.keys()))
        self.rule_book = numpy.delete(self.rule_book, self.rule_book.shape[0]-1, 0)
#         print "Fitted"
#         import pprint
#         pprint.pprint(self.rule_index)
#         print self.rule_book
        # print X_tr
        self.Bernoulli.fit(X_tr, y)
        return
            
        
        
    def fit_encoders(self, X, y):

        import numpy

        self.num_labels = len(set(y))
        self.labels = sorted(list(set(y)))
        N_samples = len(y)
        if type(X) is numpy.array:
            y = y.reshape(-1, 1)
        else:
            y = numpy.array(y).reshape(-1, 1)
        y = self.lab.fit_transform(y).reshape(-1, 1)
        y = self.ohe.fit_transform(y).todense()
        X = self.lab.transform(X)
        X = self.ohe.transform(X.T.reshape(-1, 1)).todense().reshape(N_samples, -1)
        self.num_models = int(X.shape[1] / self.num_labels)
        return X, y
    
    def transformation(self, X):
        
        import random
        #from sklearn.
        predictions = []
        #print len(X), len(y)
        for i, x in enumerate(X):
            x1 = str(x)
            if x1 in self.rule_index.keys():
#                 print "True"
#                 print y[i]
                #print self.rule_book[self.rule_index[x],:]
                #print self.rule_book[self.rule_index[x],:].argmax(axis=0)
                predictions.append(self.labels[self.rule_book[self.rule_index[x1],:].argmax(axis=0)])
#                 print "Predicted"
#                 print self.rule_book[self.rule_index[x1],:]
#                 print predictions[-1]
            else:
                predictions.append([])
                #print "Random!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
                #predictions.append(random.choice(self.labels))
                #scores = []
                #for x_ in self.transformed:
                #    pass
                #print [x]
                #print type(x)
                    # print [x[0]]
        #predictions = []
        #for x in X:
        #    predictions.append(self.Bernoulli.predict([x]))
        #predictions = self.Bernoulli.predict(X)    
        #predictions.append(self.Bernoulli.predict([]))
        return predictions
    
    
    def transform(self, X):

        import numpy

        if type(X[0]) is numpy.array:
            N_samples = X[0].shape[0]
        else:
            N_samples = len(X[0])
        X = self.lab.transform(X)
        X = self.ohe.transform(X.reshape(-1, 1)).todense().reshape(N_samples, -1)
        #print len(X), len(y)
        prediction = self.transformation(X)
        #prediction = self.lab.inverse_transform(prediction.argmax(axis=1))
        return prediction

    def predict(self, X):
        return self.transform(X)

    def score(self, X, y_true):

        from sklearn.metrics import accuracy_score

        y_pred = self.predict(X)
        return accuracy_score(y_true, y_pred, normalize=True)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class Neigbors_Combinator2(BaseEstimator, TransformerMixin):
    
    """GIving untrained Models and their transformers create a combinator of information:
       - Based on the count vectorizer predictions of a kNN classifier (fitted in X_train).
       - Based on the transformed kNN classifiers predictions (fitted in X_train)
       - Based on the samples predictions of the now trained models (fitted in X_train)
       - Combining the above predictions using weights (fitted in X_meta)
       
       Args:
           - models: dic of trained models in the form of: {"name": Model, :"name2":Model2...}
           - models_tr: dic of transformers of the models. Form like models.\
           - names: list of names for the models to be used
           - knn_params: dictionary containing params for fitting the knn classifiers.
                         Expects a dictionary like: {"general":{"num_folds":5, "n_jobs":-1},
                                                     "name":  parameter_grid_for_grid_search1,
                                                     "name2": parameter_grid_for_grid_search2,...}
            - scheme: for wegithing """
    
    def __init__(self, models, models_tr, names, knn_params, scheme='majority', weights=None):
        
        from combinations import Combinator
        from sklearn.feature_extraction.text import CountVectorizer
        
        self.counter = CountVectorizer()
        self.models = models
        self.models_tr = models_tr
        self.names = names
        self.knn_params = knn_params
        self.scheme = scheme
        self.weights = weights
        self.ind2names = {}
        for i, name in enumerate(names):
            self.ind2names[i] = name
        self.counter = CountVectorizer()
        self.gt_knn = None
        self.tr_knn = {}
        self.comb = Combinator(scheme=self.scheme)
        self.freq = Freq()
            
        

    def fit(self, X_train, X_meta, y_train, y_meta, weights=None):
        
        from sklearn.grid_search import GridSearchCV
        from sklearn.neighbors import KNeighborsClassifier

        import random, time
        #print len(y_train), len(y_meta)
        if (y_train is None)  or (y_meta is None):
            raise ValueError('we need y labels to supervise-fit!')
        else:
            parameters = {
                    'input': 'content',
                    'encoding': 'utf-8',
                    'decode_error': 'ignore',
                    'analyzer': 'word',
                    'stop_words': 'english',
                    # 'vocabulary':list(voc),
                    #'tokenizer': tokenization,
                    #'tokenizer': _twokenize.tokenizeRawTweetText,  # self.tokenization,
                    #'tokenizer': lambda text: _twokenize.tokenizeRawTweetText(nonan.sub(po_re.sub('', text))),
                    'max_df': 1.0,
                    'min_df': 1,
                    'max_features':None
                }
            
            # Fit the Meta-Classifier
            predictions_meta = []
            # Fitting the ground truth neighbors
            t0 = time.time()
            self.counter.set_params(**parameters)
            X_train_truth = self.counter.fit_transform(X_train)
            #print X_train_truth.toarray().shape
            grid_ = GridSearchCV(KNeighborsClassifier(), param_grid=self.knn_params['true'],
                                    n_jobs=self.knn_params['general']['n_jobs'], 
                                     cv=self.knn_params['general']['cv'], refit=True, 
                                      verbose=self.knn_params['general']['verbose'])
            grid_.fit(X_train_truth, y_train)
            self.gt_knn = grid_.best_estimator_
            # Add the gt predictions
            #print self.counter.transform(X_meta).toarray().shape
            predictions_meta.append(self.gt_knn.predict(self.counter.transform(X_meta)))
            # Fit the transformation Classifiers
            for name in self.names:
                X_train_tr = self.models_tr[name].transform(X_train)
                grid_ = GridSearchCV(KNeighborsClassifier(), param_grid=self.knn_params[name],
                                    n_jobs=self.knn_params['general']['n_jobs'], 
                                     cv=self.knn_params['general']['cv'], refit=True, 
                                      verbose=self.knn_params['general']['verbose'])
                grid_.fit(X_train_tr, y_train)
                self.tr_knn[name] = grid_.best_estimator_
                # Add the transformed knn predictions
                predictions_meta.append(self.tr_knn[name].predict(self.models_tr[name].transform(X_meta)))
                # Add the per sample predictions
                predictions_meta.append(self.models[name].predict(X_meta))
            #print len(predictions_meta)
            #print predictions_meta
            self.comb.fit(predictions_meta, y_meta)
            print "Best Weights Found:"
            print self.comb.weights
            print self.freq.fit(predictions_meta, y_meta)
            return
        
        
    def transform(self, X, y):

        import numpy
        
        predictions = []
        X_truth = self.counter.transform(X)   
        predictions.append(self.gt_knn.predict(X_truth))
        for name in self.names:
            #print name
            # Add the transformed knn predictions
            predictions.append(self.tr_knn[name].predict(self.models_tr[name].transform(X)))
            # Add the per sample predictions
            predictions.append(self.models[name].predict(X))
            #print len(predictions)
        #print len(predictions)
        #for pred in predictions:
        #    print len(pred)
        final_predict =self.comb.predict(predictions)
        final_predict2 = self.freq.predict(predictions)
        names = ['Ground_Truth', '3grams-Neighbors', '3grams-Pred',
                     'SOAC-Neigbors', 'SOAC-Pred', 'LSI-Neighbors', 'LSI-Pred', 'Comb']
        gg_pred = []
        for i in xrange(len(y)):
            print 'True: ' + str(y[i])
            for  j, pred in enumerate(predictions):
                print '%s: %s' % (names[j], pred[i])
            print "Comb: %s" % final_predict[i]
            #print "Freq: %s" % final_predict2[i]
            
            if final_predict2[i] != []:
                print "Freq: %s" % final_predict2[i]
                print 'Selected from Freq'
                gg_pred.append(final_predict2[i])
            else:
                gg_pred.append(final_predict[i])
            print gg_pred[-1]
            print '*'*100
        return gg_pred

    def predict(self, X, y):
        return self.transform(X, y)

    def score(self, X, y_true):

        from sklearn.metrics import accuracy_score

        y_pred = self.predict(X)
        return accuracy_score(y_true, y_pred, normalize=True)

models11 = {}
models_tr11 = {}
tr_params = {'n_neighbors':[i for i in xrange(1,20)], 'weights':['uniform', 'distance']}
knn_params = {"general":{"n_jobs":-1, "cv":5, "verbose":1}}
knn_params['true'] = tr_params
names11 = []
for i, model in enumerate(trained_base_models):
    names11.append(base_model_names[i])
    models11[names11[-1]] = model
    models_tr11[names11[-1]] = trained_base_models[i].steps[0][1]
    knn_params[names11[-1]] = tr_params
    #print base_model_names[i]
    #print trained_base_models[i].steps[0][1]
aa = Neigbors_Combinator2(models11, models_tr11, names11, knn_params, scheme='accuracy')            
aa.fit(X_train, X_meta, y_train, y_meta)
predict = aa.predict(X_cv, y_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))